# Multi-Analyst Text Analysis Pipeline

In this notebook we demonstrate the full pipeline for analyzing text through multiple specialist lenses (rhetorician, syntactician, lexicologist, etc.) and synthesizing their observations into 
* a unified writing style analysis from the text samples,
* an instruction of how to write in the analyzed style.

The workflow is *agentic* in that it involves several distinct agents built on Large Language Models (LLMs), however, the order and relation between each agent is set, not dynamically derived.

The analysis and its details are described in this blog post: **INSERT LINK**

## Installations and Preparations
External libraries are installed and tested to be in working order.

Key dependencies:
* LiteLLM (model router such that different LLM APIs can be readily employed)
* Jinja (create prompts with variables and conditional logic)
* Pydantic (create prompts from variables with validation)

**Install requirements.** Only needed if running in fresh kernel.

In [ ]:
!pip install -r requirements.txt

Check that LiteLLM was installed correctly. List the providers available via LiteLLM router.

In [ ]:
try:
    import litellm
    print('Providers\n=========')
    print('* ' + '\n* '.join(litellm.LITELLM_CHAT_PROVIDERS))
except ImportError as e:
    print(f"✗ Cannot import litellm: {e}")

## Initialize Base Objects
The base objects part of the current project library (`belletrist`) are initialized. They are:
* `LLM`: the LLM object.
* `LLMConfig`: the configuration of the LLM object, such as what model to use.
* `PromptMaker`: generates prompts from templates and variables
* `DataSampler`: retrieves and samples text at a source directory
* `ResultStore`: simple database object to save intermediate and final outputs

The LLM to use is set by the `model_string`, which is constructed as `<provider>/<model>`, the providers defined by the `litellm` package, see in particular `litellm.LITELLM_CHAT_PROVIDERS`. The API key to the provider should be stored in an environment variable with name defined in `model_provider_api_key_env_var`. You need to create that yourself for the provider of interest. 

Do **not** store the API key as a string variable directly in the notebook, you're at risk of exposing it.

In [3]:
model_string = 'mistral/mistral-large-2411'
model_provider_api_key_env_var = 'MISTRAL_API_KEY'

In [5]:
import os
from pathlib import Path
from belletrist import LLM, LLMConfig, PromptMaker, DataSampler, ResultStore

llm = LLM(LLMConfig(
    model=model_string,
    api_key=os.environ.get(model_provider_api_key_env_var)
))
prompt_maker = PromptMaker()
sampler = DataSampler(
    data_path=(Path(os.getcwd()) / "data" / "russell").resolve()
)
store = ResultStore(Path(f"{os.getcwd()}/belletrist_storage.db"))

In case a clean run is wanted, the old contents of the database are discarded with a result store reset. Do **not** run this reset if content should be preserved from previous runs. 

In [6]:
store.reset()

## Generate and Store Text Samples to be Analyzed

The `DataSampler` retrieves paragraphs from the corpus of text. The retrieval can be a random sample of consecutive paragraphs (via the method `sample_segment`) or a specific file and paragraph range (via the method `get_paragraph_chunk`).

As illustration of the process, a random four-paragraph long segment is sampled below.

In [7]:
text_sample = sampler.sample_segment(p_length=4)
print(f'Text source: {text_sample.file_path}')
print(f'Paragraph range: {text_sample.paragraph_start} - {text_sample.paragraph_end}')
print(f'\n{text_sample.text}')

Text source: /Users/andersohrn/PycharmProjects/russell_writes/data/russell/education_and_the_good_life.txt
Paragraph range: 207 - 211

CHAPTER VIII

TRUTHFULNESS

To produce the habit of truthfulness should be one of the major aims
of moral education. I do not mean truthfulness in speech only, but
also in thought; indeed, of the two, the latter seems to me the more
important. I prefer a person who lies with full consciousness of what
he is doing to a person who first subconsciously deceives himself
and then imagines that he is being virtuous and truthful. Indeed, no
man who thinks truthfully can believe that it is _always_ wrong to
speak untruthfully. Those who hold that a lie is always wrong have to
supplement this view by a great deal of casuistry and considerable
practice in misleading ambiguities, by means of which they deceive
without admitting to themselves that they are lying. Nevertheless, I
hold that the occasions when lying is justifiable are few--much fewer
than would be inf

A number of text samples are retrieved (**set `n_sample` to the desired number**) and each sample comprises a set number of paragraphs (**set `m_paragraphs_per_sample` to the desired number**). These text samples are stored in the project result stored with keys like `sample_001`, `sample_002` and so on; these keys are henceforth referring to specific text samples.

In [8]:
n_sample = 5
m_paragraphs_per_sample = 5

for _ in range(n_sample):
    sample_id = f'sample_{len(store.list_samples()) + 1:03d}'
    segment = sampler.sample_segment(p_length=m_paragraphs_per_sample)
    store.save_segment(sample_id, segment)

In [9]:
print('Sample keys:\n============')
store.list_samples()

Sample keys:


['sample_001', 'sample_002', 'sample_003', 'sample_004', 'sample_005']

## Step 1: Construct the Analyst Agents and Analyze Text Samples

Send the text samples through each specialist analyst. Each produces an independent analysis from their domain expertise.

**Prompt structure for each analyst is:**
1. Preamble instruction of task ahead 
2. Analyst-specific instruction template
3. Text to analyze

Note that the execution of this can take time since it involves invoking LLMs, once per analyst and text sample. These are however independent analyses, and can therefore *in principle* be run in parallel, though the implementation below does not utilize that fact.

Note that the agents are distinguished by their prompts, which are obtained via prompt models defined within the `bellatrist` project library.

In [11]:
from belletrist.models import (
    PreambleInstructionConfig,
    PreambleTextConfig,
    RhetoricianConfig,
    SyntacticianConfig,
    LexicologistConfig,
    InformationArchitectConfig,
    EfficiencyAuditorConfig,
    CrossPerspectiveIntegratorConfig,
)
ANALYSTS = ["rhetorician", "syntactician", "lexicologist", "information_architect", "efficiency_auditor"]
ANALYST_CONFIGS = {
    "rhetorician": RhetoricianConfig,
    "syntactician": SyntacticianConfig,
    "lexicologist": LexicologistConfig,
    "information_architect": InformationArchitectConfig,
    "efficiency_auditor": EfficiencyAuditorConfig,
}

def build_analyst_prompt(preamble_instruction: str, analyst_prompt: str, preamble_text: str) -> str:
    """
    Helper function to construct the full prompt for an analyst.
    
    """
    return f"{preamble_instruction}\n\n{analyst_prompt}\n\n{preamble_text}"

In [12]:
# Get all samples from the store
all_samples = store.list_samples()
print(f"Processing {len(all_samples)} samples with {len(ANALYSTS)} analysts each\n")

# Outer loop: iterate over each text sample
for sample_id in all_samples:
    print(f"Sample: {sample_id}")
    
    # Get the sample text
    sample = store.get_sample(sample_id)
    text = sample['text']
    
    # Build shared prompt components (reused across all analysts for this sample)
    preamble_instruction = prompt_maker.render(PreambleInstructionConfig())
    preamble_text = prompt_maker.render(PreambleTextConfig(text_to_analyze=text))
    
    # Inner loop: run each analyst on this sample
    for analyst_name in ANALYSTS:
        print(f"  Running {analyst_name}...", end=" ")
        
        # Get analyst-specific prompt using the config class
        analyst_config = ANALYST_CONFIGS[analyst_name]()
        analyst_prompt = prompt_maker.render(analyst_config)
        full_prompt = build_analyst_prompt(preamble_instruction, analyst_prompt, preamble_text)
        
        # Run analysis and save result
        response = llm.complete(full_prompt)
        store.save_analysis(sample_id, analyst_name, response.content, response.model)
        
        print(f"✓ ({len(response.content)} chars)")
    
    print()

print(f"All analyses complete for {len(all_samples)} samples")

Processing 5 samples with 5 analysts each

Sample: sample_001
  Running rhetorician... ✓ (5790 chars)
  Running syntactician... ✓ (12203 chars)
  Running lexicologist... ✓ (5372 chars)
  Running information_architect... ✓ (5967 chars)
  Running efficiency_auditor... ✓ (5428 chars)

Sample: sample_002
  Running rhetorician... ✓ (5292 chars)
  Running syntactician... ✓ (5439 chars)
  Running lexicologist... ✓ (6028 chars)
  Running information_architect... ✓ (6999 chars)
  Running efficiency_auditor... ✓ (4836 chars)

Sample: sample_003
  Running rhetorician... ✓ (5983 chars)
  Running syntactician... ✓ (10971 chars)
  Running lexicologist... ✓ (5480 chars)
  Running information_architect... ✓ (6501 chars)
  Running efficiency_auditor... ✓ (6605 chars)

Sample: sample_004
  Running rhetorician... ✓ (3853 chars)
  Running syntactician... ✓ (8259 chars)
  Running lexicologist... ✓ (6615 chars)
  Running information_architect... ✓ (8012 chars)
  Running efficiency_auditor... ✓ (5426 chars)


Verification that analysis was run as expected and yielded analysis results. Excerpt of one specific analysis retrieved from project database and printed for illustration.

In [14]:
sample_id = 'sample_001'
is_complete = store.is_complete(sample_id, ANALYSTS)
print(f"Analysis complete: {is_complete}")

# Retrieve sample and all analyses (both are now dicts)
sample, analyses = store.get_sample_with_analyses(sample_id)

print(f"\nSample: {sample['sample_id']}")
print(f"Source: File {sample['file_index']}, paragraphs {sample['paragraph_start']}-{sample['paragraph_end']}")
print(f"Analyses available: {list(analyses.keys())}")

# Examine one analysis
print(f"\n--- Rhetorician Output (first 500 chars) ---")
print(analyses.get("rhetorician", "Not found")[:500])

Analysis complete: True

Sample: sample_001
Source: File 1, paragraphs 372-377
Analyses available: ['efficiency_auditor', 'information_architect', 'lexicologist', 'rhetorician', 'syntactician']

--- Rhetorician Output (first 500 chars) ---
### RHETORICAL STRATEGY AND STANCE ANALYSIS

#### 1. WRITER'S POSITION

**Persona**: The writer emerges as authoritative but also somewhat conversational. The use of phrases like "It is of course true" and "I do not believe" suggests a confident yet approachable voice. The writer is not overly formal but maintains a clear authority on the subject.

**Relationship to Subject Matter**: The writer positions themselves as an expert, providing insights and recommendations on the educational system an


## Step 2a: Pattern Recognition (Cross-Perspective Integration) per Text Sample

Synthesize all analyst perspectives to identify interactions, tensions, and load-bearing features. This is a per-text-cross-analyst transformation. This looks to integrate multiple perspectives on each text sample and indirectly distill the information content in the assessments of the text samples. 

If only a subset of samples are to be analyzed, filter or slice the list `samples_to_analyze`, which is a list of sample IDs, as created above.

In [15]:
samples_to_analyze = store.list_samples()
print(samples_to_analyze)

['sample_001', 'sample_002', 'sample_003', 'sample_004', 'sample_005']


In [16]:
def build_pattern_prompt_from_(text: str, analyses: dict):
    """Convenience function to create the prompt, since the prompt depends on the kinds of analysts to integrate.
    
    """
    sample, analyses = store.get_sample_with_analyses(sample_id)

    analyst_info = {}
    for analyst_name in ANALYSTS:
        config_class = ANALYST_CONFIGS[analyst_name]
        analyst_info[analyst_name] = {
            'analysis': analyses[analyst_name],
            'analyst_descr_short': config_class.description()
        }

    pattern_config = CrossPerspectiveIntegratorConfig(
        original_text=sample['text'],
        analysts=analyst_info
    )
    return prompt_maker.render(pattern_config)

Note that this loop can take time to execute, since LLMs are called. Each analysis is independent and can therefore in principle be parallelized, though the implementation below does not do that.

Note also that the cross-perspective per-text result are stored in the result store, keyed on the sample ID and the analyst kind.

In [17]:
for sample_id in samples_to_analyze:
    print(f"Running Cross-Perspective Integrator agent for {sample_id}...", end=" ")
    
    sample, analyses = store.get_sample_with_analyses(sample_id)
    pattern_prompt = build_pattern_prompt_from_(text=sample['text'], analyses=analyses)

    pattern_response = llm.complete(pattern_prompt)
    
    # Store pattern recognition result in result store
    store.save_analysis(
        sample_id, 
        CrossPerspectiveIntegratorConfig.analyst_name(), 
        pattern_response.content, 
        pattern_response.model
    )
    
    print(f"✓ ({len(pattern_response.content)} chars)")

Running pattern recognizer for sample_001... ✓ (9644 chars)
Running pattern recognizer for sample_002... ✓ (9039 chars)
Running pattern recognizer for sample_003... ✓ (6755 chars)
Running pattern recognizer for sample_004... ✓ (11118 chars)
Running pattern recognizer for sample_005... ✓ (20398 chars)


In [19]:
sample_id = 'sample_005'
sample, analyses = store.get_sample_with_analyses(sample_id)
print(analyses.keys())

print(f"\n--- Pattern Analyst Output (first 2000 chars) ---")
print(analyses.get("cross_perspective_integrator", "Not found")[:2000])

dict_keys(['cross_perspective_integrator', 'efficiency_auditor', 'information_architect', 'lexicologist', 'rhetorician', 'syntactician'])

--- Pattern Analyst Output (first 2000 chars) ---
## I. Extracted Techniques

### 1. **Precise Technical Definition**
**Specification:**
Define a technical term clearly and concisely at the beginning of a paragraph. Use a single sentence for the definition, followed by an example or explanation in subsequent sentences.

**Example from text:**
"In such a law as 'A, B, C,... in the past, together with X now, cause Y now,' we will call A, B, C,... the mnemic cause, X the occasion or stimulus, and Y the reaction."

**Source observations:**
- **Lexicologist:** "Technical terms such as 'mnemic causation,' 'psycho-physical parallelism,' and 'engram' are used frequently. They are introduced directly without extensive explanation."
- **Rhetorician:** "The writer demonstrates a deep understanding of the subject and engages with complex ideas in a clear and co

## Stage 2b: Cross-Text Synthesis of Integrated Analyses

Patterns that appear across multiple text analyses are synthesized. This stage takes all the cross-perspective integration outputs and identifies overaching techniques, complementary findings, and so on, in order to construct a highly specific conclusion on the techniques that are employed in the text samples. It attempts in other words a synthesis of all analysis, across perspectives and across text samples.

This is a single document. In order to track provenance, the text samples and analyst types that went into its construction are gathered and included in the storage in the project database.

In [20]:
from belletrist.models import CrossTextSynthesizerConfig

# Get all samples that have cross-perspective integration results
all_samples = store.list_samples()
pattern_analyst = CrossPerspectiveIntegratorConfig.analyst_name()

# Retrieve all pattern recognition analyses
integrated_analyses = {}
for sample_id in all_samples:
    pattern_analysis = store.get_analysis(sample_id, pattern_analyst)
    if pattern_analysis:
        integrated_analyses[sample_id] = pattern_analysis
    else:
        print(f"⚠ Sample {sample_id} missing cross-perspective integration results")

print(f"Found {len(integrated_analyses)} cross-perspective integration results")
print(f"Sample IDs: {list(integrated_analyses.keys())}")

if len(integrated_analyses) < 2:
    print(f"\n⚠ Need at least 2 pattern recognition analyses for cross-text synthesis. Got {len(integrated_analyses)}.")

Found 5 cross-perspective integration results
Sample IDs: ['sample_001', 'sample_002', 'sample_003', 'sample_004', 'sample_005']


This is where the analysis is run. This can take time, since it requires running an LLM, however, only one LLM call in total.

In [21]:
cross_text_config = CrossTextSynthesizerConfig(
    integrated_analyses=integrated_analyses
)
cross_text_prompt = prompt_maker.render(cross_text_config)
    
print("Running Cross-Text Synthesis...", end=" ")
cross_text_response = llm.complete(cross_text_prompt)
print(f"✓ ({len(cross_text_response.content)} chars)")
    
# Save to ResultStore with auto-generated ID and full provenance
sample_contributions = [(sid, pattern_analyst) for sid in integrated_analyses.keys()]
cross_text_id = store.save_synthesis(
    synthesis_type=cross_text_config.synthesis_type(),
    output=cross_text_response.content,
    model=cross_text_response.model,
    sample_contributions=sample_contributions,
    config=cross_text_config
)
print(f"Saved as: {cross_text_id}")


Running cross-text synthesis... ✓ (11977 chars)
Saved as: cross_text_synthesis_002


In [23]:
print("\n--- Cross-Text Synthesis (first 1000 chars) ---")
print(cross_text_response.content[:2000])


--- Cross-Text Synthesis (first 1000 chars) ---
## SYNTHESIS OF INTEGRATED ANALYSES

### I. Stable Core Patterns

#### 1. **Complex Sentence Structures for Nuance**
   - **Frequency**: 4/5 texts
   - **Description**: Use complex sentences with multiple subordinate clauses to convey detailed and nuanced ideas. Main clauses should be 8-12 words, with subordinate clauses extending to 15-20 words.
   - **Examples**:
     - "We assume, that is to say, that we know what we mean by saying that a certain event is nearer to another than to a third, so that before making accurate measurements we can speak of the “neighborhood” of an event..." (Sample 003)
     - "The Far Eastern situation is so complex that it is very difficult to guess what will be the ultimate outcome of the Washington Conference, and still more difficult to know what outcome we ought to desire." (Sample 004)
   - **Centrality**: Foundational. Complex sentences allow for detailed explanations and nuanced arguments.
   - **Mec

## Stage 3: Synthesizer of Principles

The final stage converts the descriptive cross-text synthesis into actionable prescriptive writing principles. This generates a style guide that can be used to instruct an LLM to write in a similar style.

In [ ]:
from belletrist.models import SynthesizerOfPrinciplesConfig

# Get the latest cross-text synthesis
cross_text_syntheses = store.list_syntheses('cross_text_synthesis')
if cross_text_syntheses:
    # Use the most recent cross-text synthesis
    latest_cross_text_id = cross_text_syntheses[-1]
    cross_text_synthesis = store.get_synthesis(latest_cross_text_id)
    
    print(f"Using cross-text synthesis: {latest_cross_text_id}")
    
    # Build principles guide config and prompt
    principles_config = SynthesizerOfPrinciplesConfig(
        synthesis_document=cross_text_synthesis['output']
    )
    principles_prompt = prompt_maker.render(principles_config)
    
    # Run principles synthesis
    print("Running principles synthesizer...", end=" ")
    principles_response = llm.complete(principles_prompt)
    print(f"✓ ({len(principles_response.content)} chars)")
    
    # Save to ResultStore with parent linkage (inherits provenance)
    principles_id = store.save_synthesis(
        synthesis_type=principles_config.synthesis_type(),
        output=principles_response.content,
        model=principles_response.model,
        sample_contributions=[],  # Inherited from parent
        config=principles_config,
        parent_synthesis_id=latest_cross_text_id
    )
    
    print(f"Saved as: {principles_id}")
    
    # Display first part
    print("\n--- Principles Guide (first 1000 chars) ---")
    print(principles_response.content[:1000])
else:
    print("⚠ No cross-text synthesis found. Run Stage 2 first.")

## Querying Synthesis Metadata

The ResultStore tracks full provenance for all syntheses. Query metadata to understand what samples, analysts, and models contributed to each synthesis.

In [ ]:
# List all syntheses
print("All Syntheses")
print("=" * 50)
for synth_type in ['cross_text_synthesis', 'principles_guide']:
    syntheses = store.list_syntheses(synth_type)
    print(f"\n{synth_type}: {len(syntheses)} found")
    for synth_id in syntheses:
        print(f"  - {synth_id}")

# Get detailed metadata for a synthesis
if store.list_syntheses():
    print("\n\nDetailed Metadata Example")
    print("=" * 50)
    
    # Get first principles guide (or first cross-text if none)
    principles = store.list_syntheses('principles_guide')
    if principles:
        synth_id = principles[0]
    else:
        synth_id = store.list_syntheses()[0]
    
    synth_with_meta = store.get_synthesis_with_metadata(synth_id)
    
    print(f"\nSynthesis ID: {synth_with_meta['synthesis_id']}")
    print(f"Type: {synth_with_meta['type']}")
    print(f"Model: {synth_with_meta['model']}")
    print(f"Created: {synth_with_meta['created_at']}")
    print(f"Parent: {synth_with_meta['parent_id']}")
    
    if synth_with_meta.get('metadata'):
        meta = synth_with_meta['metadata']
        print(f"\nMetadata:")
        print(f"  Samples: {meta['num_samples']}")
        print(f"  Sample IDs: {meta['sample_ids']}")
        print(f"  Model homogeneous: {meta['is_homogeneous_model']}")
        print(f"  Models used: {meta['models_used']}")

# Get full provenance tree
if store.list_syntheses('principles_guide'):
    print("\n\nFull Provenance Tree")
    print("=" * 50)
    
    principles_id = store.list_syntheses('principles_guide')[0]
    provenance = store.get_synthesis_provenance(principles_id)
    
    print(f"\nPrinciples Guide: {provenance['synthesis']['synthesis_id']}")
    print(f"  Created: {provenance['synthesis']['created_at']}")
    print(f"  Model: {provenance['synthesis']['model']}")
    
    if provenance['parent']:
        parent = provenance['parent']
        print(f"\n  Parent (Cross-Text): {parent['synthesis']['synthesis_id']}")
        print(f"    Sample contributions: {len(parent['sample_contributions'])}")
        for sample_id, analyst in parent['sample_contributions'][:3]:
            print(f"      - {sample_id} / {analyst}")
        if len(parent['sample_contributions']) > 3:
            print(f"      ... and {len(parent['sample_contributions']) - 3} more")

## Exporting Syntheses to Filesystem

Export final syntheses to text files with YAML metadata headers for consumption by other tools or LLMs.

In [ ]:
# Create outputs directory
outputs_dir = Path("outputs")
outputs_dir.mkdir(exist_ok=True)

# Export cross-text synthesis
cross_text_syntheses = store.list_syntheses('cross_text_synthesis')
if cross_text_syntheses:
    for synth_id in cross_text_syntheses:
        output_path = outputs_dir / f"{synth_id}.txt"
        store.export_synthesis(synth_id, output_path, metadata_format='yaml')
        print(f"Exported: {output_path}")

# Export principles guide
principles_guides = store.list_syntheses('principles_guide')
if principles_guides:
    for synth_id in principles_guides:
        output_path = outputs_dir / f"{synth_id}.txt"
        store.export_synthesis(synth_id, output_path, metadata_format='yaml')
        print(f"Exported: {output_path}")
        
        # Also create a special "derived_style_instructions.txt" for style_evaluation.ipynb
        if synth_id == principles_guides[-1]:  # Use latest
            instructions_path = outputs_dir / "derived_style_instructions.txt"
            store.export_synthesis(synth_id, instructions_path, metadata_format='yaml')
            print(f"Exported for style evaluation: {instructions_path}")

print(f"\nAll syntheses exported to {outputs_dir.absolute()}")

## Utilities: Working with Stored Samples

Helper functions for browsing and managing stored results.

In [ ]:
# List all samples in the database
all_samples = store.list_samples()
print(f"Total samples: {len(all_samples)}")
print(f"Sample IDs: {all_samples}")

# Check completion status for each
print("\nCompletion status:")
for sid in all_samples:
    complete = store.is_complete(sid, ANALYSTS)
    status = "✓" if complete else "✗"
    print(f"  {status} {sid}")

# Close database connection when done
# store.close()